<a href="https://colab.research.google.com/github/youve-been/sogang/blob/main/Capstone/backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 서버 실행 전에 실행해야할 셀 모음

In [ ]:
# 드라이브 연동

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 라이브러리 설치
!apt-get update
!apt-get install -y locales
!locale-gen en_US.UTF-8
!update-locale LANG=en_US.UTF-8
!apt-get install -y ffmpeg

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# whisper 설치 - 음성인식 모델
!pip install -U openai-whisper
# pyannote 설치 - 화자 분리를 위한 API(metrics - 화자분리 정확도 평가)
!pip install pyannote.audio
!pip install pyannote.metrics
!pip install pydub
# jiwer 설치 - 음성인식 정확도 평가(WER/CER)를 위한 라이브러리
!pip install jiwer
# openai 설치 - gpt 사용
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet
# word 파일 만들기
!pip install python-docx
# fastapi, uvicorn, nest-asyncio, pyngrok 설치 - 백엔드 서버 열기
!pip install fastapi uvicorn nest-asyncio pyngrok
# ngrok 토큰 인증 ** 토큰 입력 필요 **
!ngrok authtoken 2pa29fRMwuW1bXLItmPWhkvPSIT_4At9v2oU7WFnsFjxKmTZw


!apt-get install -y ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,196 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,543 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,458 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubunt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.1 MB/s eta 0:00:00
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.7 MB/s eta 0:00:00


# 백엔드 서버 코드

In [ ]:
import sys

GOOGLE_DRIVE_PATH = "/content/drive/MyDrive/backendpython" # .py 코드들이 저장되어 있는 폴더 path 입력
sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
from fastapi import FastAPI, responses, HTTPException, Request
from fastapi.responses import HTMLResponse
import uvicorn
from pyngrok import ngrok
import nest_asyncio
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List
from typing import Dict
from google.colab import files
#-------------------------------------------------------------------------------------------------
#--------- 백엔드 파이썬 파일  -------------------------------------------------------------------
from word import add_custom_text_to_adjacent_empty_cells
from stt import split_files, delete_files, diarization, transcribe
from gpt import use_gpt, summarize

from gpt import summary
#-------------------------------------------------------------------------------------------------

app = FastAPI()
origins = [
    "http://localhost:3000",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class TextUpdate(BaseModel):
    id: int
    item: str

audio_file="test.wav"

#-------------------------------------------------------------------------------------------------
#-------  전사한 내용 출력(Convert/Text.js)  -----------------------------------------------------
@app.get("/convert")
async def root():
    with open('/content/drive/MyDrive/MeetingAudio/STT.txt', 'r', encoding='utf-8') as f:
      text = f.read()
    return text
#-------------------------------------------------------------------------------------------------

#-------------------------------------------------------------------------------------------------
#-------  음성인식 시작(Home/.UploadFile.js)  ----------------------------------------------------
@app.get("/create")
async def create_clicked():
  diarization(audio_file)
  split_files(audio_file)
  transcribe(audio_file)
  delete_files('/content/splittedfiles')
  summarize('/content/drive/MyDrive/MeetingAudio/STT.txt')
  return {"message": "Welcome to your todo list."}
#-------------------------------------------------------------------------------------------------

@app.get("/summary",tags=["summary"])
async def getsummary()->dict:
  return {"data": summary}

@app.put("/summary/{text_id}")
def update_text(text_id: int, text_update: TextUpdate):
    for text in summary:
        if text["id"] == text_id:
            text["item"] = text_update.item
            return {"message": "Text updated successfully", "data": text}
    raise HTTPException(status_code=404, detail="Text not found")


#-------------------------------------------------------------------------------------------------
#-------  WORD 파일 생성(Summary/Text.js)  -------------------------------------------------------
output_path_template2_with_custom_content = '/content/drive/MyDrive/Meeting Report.docx'
docx_path_template2 ='/content/drive/MyDrive/Meeting Report template.docx'

class DictionaryModel(BaseModel):
    data: Dict[str, str]  # 딕셔너리 타입

@app.post("/receive-dictionary")
def receive_dictionary(dictionary: DictionaryModel):
    add_custom_text_to_adjacent_empty_cells(docx_path_template2, output_path_template2_with_custom_content, dictionary.data)
    print("Received dictionary:", dictionary.data)
    files.download(output_path_template2_with_custom_content)
    return {"message": "Dictionary received successfully", "received_data": dictionary.data}
#-------------------------------------------------------------------------------------------------

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://2d6c-34-125-22-126.ngrok-free.app


INFO:     Started server process [22753]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     180.64.1.14:0 - "OPTIONS /summary HTTP/1.1" 200 OK
INFO:     180.64.1.14:0 - "GET /summary HTTP/1.1" 200 OK
INFO:     180.64.1.14:0 - "OPTIONS /convert HTTP/1.1" 200 OK
INFO:     180.64.1.14:0 - "GET /convert HTTP/1.1" 200 OK
INFO:     180.64.1.14:0 - "GET /summary HTTP/1.1" 200 OK
INFO:     180.64.1.14:0 - "OPTIONS /summary1 HTTP/1.1" 200 OK
INFO:     180.64.1.14:0 - "OPTIONS /summary1 HTTP/1.1" 200 OK
INFO:     180.64.1.14:0 - "OPTIONS /summary1 HTTP/1.1" 200 OK
INFO:     180.64.1.14:0 - "PUT /summary1 HTTP/1.1" 404 Not Found
INFO:     180.64.1.14:0 - "PUT /summary1 HTTP/1.1" 404 Not Found
INFO:     180.64.1.14:0 - "PUT /summary1 HTTP/1.1" 404 Not Found
INFO:     180.64.1.14:0 - "PUT /summary1 HTTP/1.1" 404 Not Found
INFO:     180.64.1.14:0 - "PUT /summary1 HTTP/1.1" 404 Not Found
INFO:     180.64.1.14:0 - "PUT /summary1 HTTP/1.1" 404 Not Found
INFO:     180.64.1.14:0 - "PUT /summary1 HTTP/1.1" 404 Not Found
INFO:     180.64.1.14:0 - "PUT /summary1 HTTP/1.1" 404 Not Found
I

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

INFO:     180.64.1.14:0 - "POST /receive-dictionary HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [22753]





# 테스트 코드

In [ ]:
# 디버깅용
%load_ext autoreload
%autoreload 2

In [ ]:
#디버깅용
%reload_ext autoreload

In [ ]:
import sys

GOOGLE_DRIVE_PATH = "/content/drive/MyDrive/CAPSTONE2" # 코드가 저장되어 있는 폴더 path 입력
sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
from stt import hello_stt
from gpt import hello_gpt
from word import hello_word
# 맞는 버전의 코드를 사용중인지 테스트 - 1.6가 최신
# 계속 이전 버전으로 나온다면 런타임 - 세션 다시 시작 후 위 3개의 셀을 다시 실행한 후 이 셀을 실행할 것
hello_stt()
hello_gpt()
hello_word()

Hello, World! version 1.6
Hello, World! version 1.6
Hello, World! version 1.6


In [ ]:
# stt, gpt 사용방법

from stt import diarization, transcribe
from gpt import summarize

filename = "test.wav"

diarization(filename)
fulltext_filename = transcribe(filename)

summary = summarize(fulltext_filename)

/usr/local/lib/python3.10/dist-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle modul

모든 파일이 /content/splittedfiles에서 삭제되었습니다.
[{'id': 0, 'name': '안건', 'item': ['마약 확산 문제와 청소년 약물 남용에 대한 사회적 대책 논의.']}, {'id': 1, 'name': '일시', 'item': ''}, {'id': 2, 'name': '책임자명', 'item': ''}, {'id': 3, 'name': '장소', 'item': ''}, {'id': 4, 'name': '참석인원', 'item': ''}, {'id': 5, 'name': '주요 안건', 'item': ['마약 문제의 심각성과 확산 방지를 위한 대책 논의.']}, {'id': 6, 'name': '회의 내용', 'item': ['회의에서는 다양한 주제들이 논의되었습니다. 먼저, 멀티팩과 관련된 대화가 있었고, 이어서 한국인들의 위가 헬리코박터 파일로리균에 노출되어 있다는 건강 문제에 대한 언급이 있었습니다. 이후 대한민국 경제와 관련된 논의가 진행되었으며, 사회 지도층의 마약 문제와 그 확산에 대한 심각성이 강조되었습니다. 특히, 마약의 사회적 폐해와 청소년들에게 미치는 영향에 대한 우려가 제기되었고, 이를 해결하기 위한 예방 교육과 치료 시스템의 필요성이 강조되었습니다. 또한, 마약 중독자를 범죄자가 아닌 환자로 보고 치료해야 한다는 의견과 함께, 법적 제재와 치료의 균형을 맞추는 것이 중요하다는 논의가 있었습니다. 마지막으로, 마약 문제 해결을 위한 정부의 역할과 사회적 인식 변화의 필요성이 강조되었습니다.']}, {'id': 7, 'name': '회의 결과', 'item': ['회의에서는 마약 문제의 심각성을 논의하며, 예방 교육 강화, 치료 및 재활 시스템 확충, 법적 제재와 치료 병행, 사회적 인식 개선 등의 대책이 필요하다는 결론을 내렸다.']}]


/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py:184: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if not hasattr(module, '__file__') or module.__file__ is None:



# import 오류 시에 한 번만 돌리시오

In [ ]:
import os
os.kill(os.getpid(), 9)